# �� **Módulo 8: Projetos Práticos - Colocando a Mão na Massa**

## **Aula 8.1: Projeto 1 - Chatbot para E-commerce**

---

### **Tá, mas o que vamos construir?**

Vamos criar um **chatbot inteligente para e-commerce** que:

- 🛍️ **Conhece todos os produtos**
- �� **Atende clientes 24/7**
- 🧠 **Lembra das preferências**
- 🔍 **Busca produtos inteligentemente**
- 💳 **Ajuda com compras**

É como ter um **vendedor digital super inteligente** que nunca dorme e conhece cada cliente! 🛒

### **Por que este projeto é importante?**

**Problema real**: E-commerces perdem vendas por falta de atendimento
**Solução**: Chatbot que atende instantaneamente e converte vendas

É como a diferença entre **uma loja fechada** e **uma loja com vendedor 24h**! 🏪

---

**🖼️ Sugestão de imagem**: Um chatbot de e-commerce atendendo clientes com produtos ao redor

### **Setup Inicial - Preparando o Terreno**

In [ ]:
# Importando bibliotecas para o projeto
import os
import json
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

# Carregando variáveis
load_dotenv()

# Modelo
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.7,
    api_key=os.getenv('OPENAI_API_KEY')  # Configure sua API key no Colab
)

print("🛍️ Setup completo para o projeto de E-commerce!")
print(f"�� Modelo: {llm.model_name}")

### **Criando Base de Produtos**

Vamos criar uma base de produtos para nossa loja virtual:

In [ ]:
# Criando base de produtos da loja
# Como criar um catálogo digital

produtos_ecommerce = """
CATÁLOGO DE PRODUTOS - TECHSTORE

SMARTPHONES:

iPhone 15 Pro Max
- Preço: R$ 9.999
- Categoria: Smartphone
- Marca: Apple
- Características: Tela 6.7", Chip A17 Pro, Câmera tripla 48MP
- Cores: Titânio Natural, Titânio Azul, Titânio Branco, Titânio Preto
- Capacidades: 256GB, 512GB, 1TB
- Garantia: 1 ano

Samsung Galaxy S24 Ultra
- Preço: R$ 8.999
- Categoria: Smartphone
- Marca: Samsung
- Características: Tela 6.8", Snapdragon 8 Gen 3, Câmera quádrupla 200MP
- Cores: Titânio Cinza, Titânio Preto, Titânio Violeta, Titânio Amarelo
- Capacidades: 256GB, 512GB, 1TB
- Garantia: 1 ano

Google Pixel 8 Pro
- Preço: R$ 7.499
- Categoria: Smartphone
- Marca: Google
- Características: Tela 6.7", Tensor G3, Câmera tripla 50MP
- Cores: Obsidian, Porcelain, Bay
- Capacidades: 128GB, 256GB, 512GB
- Garantia: 1 ano

NOTEBOOKS:

MacBook Pro 14" M3 Pro
- Preço: R$ 18.999
- Categoria: Notebook
- Marca: Apple
- Características: Chip M3 Pro, 14", 18GB RAM, 512GB SSD
- Cores: Space Gray, Silver
- Garantia: 1 ano

Dell XPS 13 Plus
- Preço: R$ 12.999
- Categoria: Notebook
- Marca: Dell
- Características: Intel i7-1360P, 13.4", 16GB RAM, 512GB SSD
- Cores: Platinum Silver, Graphite
- Garantia: 1 ano

ACESSÓRIOS:

AirPods Pro 2ª Geração
- Preço: R$ 2.499
- Categoria: Fones de Ouvido
- Marca: Apple
- Características: Cancelamento de ruído ativo, Áudio espacial
- Cores: Branco
- Garantia: 1 ano

Apple Watch Series 9
- Preço: R$ 3.999
- Categoria: Smartwatch
- Marca: Apple
- Características: Tela 45mm, Chip S9, Monitor cardíaco
- Cores: Midnight, Starlight, Pink, Blue, Red
- Garantia: 1 ano

POLÍTICAS DA LOJA:

ENTREGA:
- Frete grátis para compras acima de R$ 500
- Entrega em 1-3 dias úteis
- Rastreamento em tempo real

PAGAMENTO:
- Cartão de crédito (até 12x)
- PIX (5% de desconto)
- Boleto bancário

GARANTIA:
- Todos os produtos têm garantia de fábrica
- Suporte técnico especializado
- Troca em até 7 dias

ATENDIMENTO:
- Chat 24/7
- WhatsApp: (11) 99999-9999
- Email: atendimento@techstore.com
"""

# Salvando base de produtos
with open('produtos_techstore.txt', 'w', encoding='utf-8') as file:
    file.write(produtos_ecommerce)

print("🛍️ Base de produtos criada!")
print("�� Contém smartphones, notebooks e acessórios da TechStore")

### **Criando o Chatbot de E-commerce**

Agora vamos criar o chatbot que conhece todos os produtos:

In [ ]:
# Criando chatbot de e-commerce
# Como criar um vendedor digital super inteligente

# Carregando e processando base de produtos
from langchain.document_loaders import TextLoader

loader = TextLoader('produtos_techstore.txt', encoding='utf-8')
documentos = loader.load()

# Dividindo em pedaços menores
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
textos_divididos = text_splitter.split_documents(documentos)

print(f"📄 Base de produtos dividida em {len(textos_divididos)} pedaços")

# Criando vector store
embeddings = OpenAIEmbeddings(
    openai_api_key=os.getenv('OPENAI_API_KEY')  # Configure sua API key no Colab
)

vectorstore = Chroma.from_documents(
    documents=textos_divididos,
    embedding=embeddings,
    collection_name="produtos_techstore"
)

print("🧠 Vector store de produtos criado!")

In [ ]:
# Criando memory para o chatbot
# Para lembrar das preferências do cliente

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# Criando chatbot com RAG
from langchain.chains import ConversationalRetrievalChain

chatbot_ecommerce = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    memory=memory,
    return_source_documents=True,
    verbose=True
)

print("🤖 Chatbot de E-commerce criado!")
print("🛍️ Vendedor digital pronto para atender!")

In [ ]:
# Testando o chatbot de e-commerce
# Vamos ver o vendedor digital em ação!

print("��️ TESTE: CHATBOT DE E-COMMERCE")
print("=" * 60)

conversas_teste = [
    "Olá! Estou procurando um smartphone bom para fotos.",
    "Qual a diferença entre iPhone 15 Pro Max e Samsung Galaxy S24 Ultra?",
    "Quanto custa o iPhone 15 Pro Max?",
    "Vocês têm frete grátis?",
    "Lembra do que eu estava procurando?",
    "Quais são as formas de pagamento?",
    "Preciso de um notebook para trabalho. O que você recomenda?"
]

for i, mensagem in enumerate(conversas_teste, 1):
    print(f"\n�� Cliente {i}: {mensagem}")
    
    try:
        # Executando chatbot
        resultado = chatbot_ecommerce({"question": mensagem})
        
        print(f"🤖 Vendedor: {resultado['answer'][:300]}...")
        
        if resultado['source_documents']:
            print(f"📄 Consultou {len(resultado['source_documents'])} produtos")
        
    except Exception as e:
        print(f"❌ Erro: {e}")
    
    print("-" * 40)

## **Aula 8.2: Projeto 2 - Assistente de Estudos**

### **O que vamos construir?**

Vamos criar um **assistente de estudos inteligente** que:

- 📚 **Analisa documentos acadêmicos**
- 📝 **Gera resumos automáticos**
- 🎴 **Cria flashcards**
- ❓ **Sistema de perguntas e respostas**
- 📊 **Acompanha o progresso**

É como ter um **professor particular 24h** que nunca se cansa! 👨‍🏫

### **Por que este projeto é útil?**

**Problema real**: Estudantes têm dificuldade para organizar e revisar material
**Solução**: Assistente que automatiza tarefas de estudo

É como a diferença entre **estudar sozinho** e **ter um tutor personalizado**! 📖

In [ ]:
# Criando base de conhecimento acadêmica
# Como criar uma biblioteca de estudos

material_estudo = """
INTRODUÇÃO À INTELIGÊNCIA ARTIFICIAL

DEFINIÇÃO E CONCEITOS BÁSICOS:

Inteligência Artificial (IA) é um campo da computação que busca criar sistemas
capazes de realizar tarefas que normalmente requerem inteligência humana.
A IA combina ciência da computação, matemática, psicologia e outras disciplinas.

HISTÓRICO DA IA:

A IA nasceu na década de 1950, com o teste de Turing proposto por Alan Turing.
O termo "Inteligência Artificial" foi cunhado em 1956 na Conferência de Dartmouth.
Desde então, a IA passou por vários "invernos" e "primaveras" de desenvolvimento.

TIPOS DE INTELIGÊNCIA ARTIFICIAL:

1. IA Fraca (Narrow AI):
   - Especializada em uma tarefa específica
   - Exemplos: Siri, Alexa, sistemas de recomendação
   - Não possui consciência ou inteligência geral

2. IA Forte (AGI - Artificial General Intelligence):
   - Inteligência geral como humanos
   - Pode realizar qualquer tarefa intelectual
   - Ainda não foi alcançada

3. Superinteligência:
   - Inteligência superior à humana
   - Conceito teórico e controverso

MACHINE LEARNING:

Machine Learning é um subcampo da IA que permite aos computadores aprender
sem serem explicitamente programados. Os algoritmos identificam padrões
nos dados para fazer previsões ou tomar decisões.

TIPOS DE MACHINE LEARNING:

1. Aprendizado Supervisionado:
   - Usa dados rotulados para treinar
   - Exemplos: classificação, regressão
   - Algoritmos: Random Forest, SVM, Redes Neurais

2. Aprendizado Não Supervisionado:
   - Encontra padrões em dados não rotulados
   - Exemplos: clustering, redução de dimensionalidade
   - Algoritmos: K-means, PCA, Autoencoders

3. Aprendizado por Reforço:
   - Aprende através de tentativa e erro
   - Usa recompensas e punições
   - Exemplos: jogos, robótica, carros autônomos

DEEP LEARNING:

Deep Learning é um subcampo do Machine Learning que usa redes neurais
com múltiplas camadas para aprender representações complexas dos dados.

CARACTERÍSTICAS:
- Múltiplas camadas de neurônios
- Aprendizado de features automático
- Requer grandes quantidades de dados
- Poder computacional significativo

APLICAÇÕES PRÁTICAS:

1. Visão Computacional:
   - Reconhecimento de imagens
   - Detecção de objetos
   - Processamento de vídeo

2. Processamento de Linguagem Natural:
   - Tradução automática
   - Análise de sentimentos
   - Chatbots inteligentes

3. Sistemas de Recomendação:
   - Netflix, Spotify, Amazon
   - Personalização de conteúdo
   - Marketing direcionado

4. Medicina e Saúde:
   - Diagnóstico por imagem
   - Descoberta de medicamentos
   - Medicina personalizada

DESAFIOS E LIMITAÇÕES:

1. Viés nos Dados:
   - Dados podem refletir preconceitos
   - Importante para justiça e equidade

2. Interpretabilidade:
   - Modelos complexos são difíceis de explicar
   - Importante para confiança e responsabilidade

3. Privacidade:
   - Coleta e uso de dados pessoais
   - Necessidade de proteção

4. Segurança:
   - Vulnerabilidades em sistemas de IA
   - Ataques adversariais

FUTURO DA IA:

O futuro da IA inclui:
- IA mais ética e responsável
- Automação de mais tarefas
- Integração com IoT e robótica
- Desenvolvimento de AGI (controverso)
- Regulamentações e políticas

CONCEITOS IMPORTANTES:

Overfitting: Modelo que memoriza dados de treino
Underfitting: Modelo muito simples para os dados
Cross-validation: Técnica para avaliar modelos
Feature Engineering: Criação de features relevantes
Hyperparameter Tuning: Otimização de parâmetros
"""

# Salvando material de estudo
with open('material_ia.txt', 'w', encoding='utf-8') as file:
    file.write(material_estudo)

print("�� Material de estudo criado!")
print("🧠 Contém conteúdo completo sobre Inteligência Artificial")

In [ ]:
# Criando assistente de estudos
# Como criar um professor particular digital

# Carregando material de estudo
loader = TextLoader('material_ia.txt', encoding='utf-8')
documentos = loader.load()

# Dividindo em pedaços menores
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150
)
textos_divididos = text_splitter.split_documents(documentos)

print(f"📄 Material dividido em {len(textos_divididos)} seções")

# Criando vector store para estudos
vectorstore_estudos = Chroma.from_documents(
    documents=textos_divididos,
    embedding=embeddings,
    collection_name="estudos_ia"
)

print("🧠 Vector store de estudos criado!")

In [ ]:
# Criando funções especializadas do assistente
# Como dar superpoderes para o professor digital

def gerar_resumo(texto, llm):
    """Gera resumo do material de estudo"""
    prompt = f"""
    Crie um resumo conciso e didático do seguinte texto sobre IA:
    
    {texto}
    
    O resumo deve:
    - Ser fácil de entender
    - Destacar os pontos principais
    - Usar linguagem clara
    - Ter no máximo 200 palavras
    """
    
    try:
        response = llm.invoke([HumanMessage(content=prompt)])
        return response.content
    except Exception as e:
        return f"Erro ao gerar resumo: {e}"

def criar_flashcards(texto, llm):
    """Cria flashcards para estudo"""
    prompt = f"""
    Crie 5 flashcards para estudo baseados no seguinte texto:
    
    {texto}
    
    Formato:
    Frente: [Pergunta]
    Verso: [Resposta]
    
    As perguntas devem ser claras e as respostas concisas.
    """
    
    try:
        response = llm.invoke([HumanMessage(content=prompt)])
        return response.content
    except Exception as e:
        return f"Erro ao criar flashcards: {e}"

def gerar_perguntas(texto, llm):
    """Gera perguntas de revisão"""
    prompt = f"""
    Crie 3 perguntas de revisão sobre o seguinte conteúdo:
    
    {texto}
    
    As perguntas devem:
    - Testar compreensão
    - Ser de diferentes níveis (fácil, médio, difícil)
    - Ter respostas claras
    """
    
    try:
        response = llm.invoke([HumanMessage(content=prompt)])
        return response.content
    except Exception as e:
        return f"Erro ao gerar perguntas: {e}"

print("🎓 Funções do assistente criadas!")
print("�� Pode gerar resumos, flashcards e perguntas")

In [ ]:
# Testando o assistente de estudos
# Vamos ver o professor digital em ação!

print("🎓 TESTE: ASSISTENTE DE ESTUDOS")
print("=" * 50)

# Buscando material sobre Machine Learning
docs_ml = vectorstore_estudos.similarity_search("Machine Learning", k=2)
texto_ml = "\n\n".join([doc.page_content for doc in docs_ml])

print(f"📚 Material encontrado sobre Machine Learning ({len(texto_ml)} caracteres)")
print("=" * 50)

# Teste 1: Gerando resumo
print("\n📝 GERANDO RESUMO:")
resumo = gerar_resumo(texto_ml, llm)
print(f"📋 Resumo:\n{resumo}")

print("\n" + "-" * 40)

# Teste 2: Criando flashcards
print("\n�� CRIANDO FLASHCARDS:")
flashcards = criar_flashcards(texto_ml, llm)
print(f"🎴 Flashcards:\n{flashcards}")

print("\n" + "-" * 40)

# Teste 3: Gerando perguntas
print("\n❓ GERANDO PERGUNTAS DE REVISÃO:")
perguntas = gerar_perguntas(texto_ml, llm)
print(f"❓ Perguntas:\n{perguntas}")

print("\n" + "=" * 50)
print("🎓 Assistente de estudos funcionando perfeitamente!")

### **Sistema Completo de Estudos**

Agora vamos criar um **sistema completo** que combina tudo:

In [ ]:
# Criando sistema completo de estudos
# Como criar um tutor personalizado completo

class AssistenteEstudos:
    def __init__(self, llm, vectorstore):
        self.llm = llm
        self.vectorstore = vectorstore
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )
    
    def perguntar(self, pergunta):
        """Faz uma pergunta sobre o material"""
        try:
            # Buscando material relevante
            docs = self.vectorstore.similarity_search(pergunta, k=3)
            contexto = "\n\n".join([doc.page_content for doc in docs])
            
            # Gerando resposta
            prompt = f"""
            Você é um professor especialista em IA. Responda à pergunta do aluno
            usando o seguinte material de estudo:
            
            MATERIAL: {contexto}
            
            PERGUNTA: {pergunta}
            
            Responda de forma didática e clara, como o Pedro Guth explicando.
            """
            
            response = self.llm.invoke([HumanMessage(content=prompt)])
            return response.content
            
        except Exception as e:
            return f"Erro: {e}"
    
    def gerar_resumo_tema(self, tema):
        """Gera resumo de um tema específico"""
        try:
            docs = self.vectorstore.similarity_search(tema, k=3)
            texto = "\n\n".join([doc.page_content for doc in docs])
            return gerar_resumo(texto, self.llm)
        except Exception as e:
            return f"Erro: {e}"
    
    def criar_flashcards_tema(self, tema):
        """Cria flashcards de um tema"""
        try:
            docs = self.vectorstore.similarity_search(tema, k=2)
            texto = "\n\n".join([doc.page_content for doc in docs])
            return criar_flashcards(texto, self.llm)
        except Exception as e:
            return f"Erro: {e}"

# Criando instância do assistente
assistente = AssistenteEstudos(llm, vectorstore_estudos)

print("🎓 Sistema completo de estudos criado!")
print("👨‍🏫 Tutor personalizado pronto para ajudar!")

In [ ]:
# Testando o sistema completo
# Vamos ver o tutor em ação!

print("🎓 TESTE: SISTEMA COMPLETO DE ESTUDOS")
print("=" * 60)

# Teste 1: Pergunta sobre Deep Learning
print("\n❓ PERGUNTA: O que é Deep Learning?")
resposta = assistente.perguntar("O que é Deep Learning e como funciona?")
print(f"��‍🏫 Professor: {resposta[:400]}...")

print("\n" + "-" * 40)

# Teste 2: Resumo sobre aplicações
print("\n�� RESUMO: Aplicações práticas de IA")
resumo = assistente.gerar_resumo_tema("aplicações práticas")
print(f"📋 Resumo:\n{resumo}")

print("\n" + "-" * 40)

# Teste 3: Flashcards sobre tipos de IA
print("\n🎴 FLASHCARDS: Tipos de Inteligência Artificial")
flashcards = assistente.criar_flashcards_tema("tipos de inteligência artificial")
print(f"🎴 Flashcards:\n{flashcards}")

print("\n" + "=" * 60)
print("�� Sistema de estudos funcionando perfeitamente!")

### **Na Prática, Meu Consagrado!** 💪

**O que construímos nos Projetos Práticos:**

1. ✅ **Chatbot de E-commerce** - Vendedor digital 24/7
2. ✅ **Assistente de Estudos** - Professor particular digital
3. ✅ **Sistemas Completos** - Funcionais e prontos para uso
4. ✅ **Aplicação Real** - Problemas reais, soluções práticas

### **Vantagens dos Projetos**

- **Escalabilidade**: Podem atender milhares de usuários
- **Disponibilidade**: Funcionam 24/7
- **Personalização**: Adaptam-se a cada usuário
- **Eficiência**: Automatizam tarefas complexas

### **Próximos Passos**

**🖼️ Sugestão de imagem**: Screenshots dos projetos funcionando

**🎯 Próximo módulo**: Vamos aprender sobre **Deploy** - como colocar os projetos na rua!

**�� Resumo do Módulo 8**:
- ✅ Chatbot de E-commerce completo
- ✅ Assistente de Estudos inteligente
- ✅ Sistemas funcionais e práticos
- ✅ Aplicação real dos conceitos

**🚀 Agora você tem projetos reais e funcionais!**